In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB7,EfficientNetB6, DenseNet201, InceptionV3, ResNet101
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your training and validation data
train_data_dir = 'NLM Augmentation/train'
validation_data_dir = 'NLM Augmentation/test'


In [2]:
# Set the image size and batch size
img_width, img_height = 224, 224
batch_size = 16

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Only rescale for validation data (no data augmentation)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and augment the training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and rescale the validation data
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 6951 images belonging to 7 classes.
Found 3001 images belonging to 7 classes.


In [3]:
# Define the number of classes in your dataset
num_classes = len(train_generator.class_indices)

# Function to create individual transfer learning models
def create_transfer_learning_model(base_model, img_width, img_height):
    # Load the base model (without the top classification layers)
    base_model = base_model(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    
    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Add a new classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Create the transfer learning model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create individual models using transfer learning
efficientnet_b6_model = create_transfer_learning_model(EfficientNetB6, img_width, img_height)
densenet_model = create_transfer_learning_model(DenseNet201, img_width, img_height)
inception_model = create_transfer_learning_model(InceptionV3, img_width, img_height)
resnet101_model = create_transfer_learning_model(ResNet101, img_width, img_height)
efficientnet_b7_model = create_transfer_learning_model(EfficientNetB7, img_width, img_height)


171446536/171446536 [==============================] - 30s 0us/step


In [ ]:
# Set the number of training and validation samples
num_train_samples = len(train_generator.filenames)
num_val_samples = len(validation_generator.filenames)

# Set the number of training epochs
epochs = 10

# Train individual models
efficientnet_b6_model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_val_samples // batch_size
)
# Save the model
efficientnet_b6_model.save('efficientnet_b6_model')

densenet_model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_val_samples // batch_size
)
# Save the model
densenet_model.save('densenet_model')

inception_model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_val_samples // batch_size
)
# Save the model
inception_model.save('inception_model')

resnet101_model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_val_samples // batch_size
)
# Save the model
resnet101_model.save('resnet101_model')

efficientnet_b7_model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_val_samples // batch_size
)
# Save the model
efficientnet_b7_model.save('efficientnet_b7_model')


In [ ]:
# Function to perform ensemble prediction using majority voting
def ensemble_prediction(models, data_generator):
    predictions = []
    for model in models:
        preds = model.predict(data_generator, verbose=1)
        predictions.append(preds)
    return np.mean(predictions, axis=0)

# Prepare the ensemble of models
ensemble_models = [efficientnet_b6_model, densenet_model, inception_model, resnet101_model, efficientnet_b7_model]

# Perform ensemble prediction on the validation data
ensemble_validation_predictions = ensemble_prediction(ensemble_models, validation_generator)

# Evaluate the ensemble performance
ensemble_validation_loss, ensemble_validation_accuracy = ensemble_models[0].evaluate(validation_generator)
print("Ensemble Validation Accuracy:", ensemble_validation_accuracy)
